In [2]:
import pandas as pd
import numpy as np

In [3]:
file_path = "auto-mpg.csv"

data = pd.read_csv(file_path)

In [ ]:
data.head()


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [ ]:
#csv dataset error fix
for index, value in enumerate(data["horsepower"]):
        if value == "?":  #check if the value is "?"
            data.at[index, "horsepower"] = None  #replace for NaN
            data = data.dropna() #deletes lines that contain NaN
data["horsepower"] = pd.to_numeric(data["horsepower"], errors='coerce')

In [ ]:
#using sckit-learn to split the dataset
'''from sklearn.model_selection import train_test_split

X = data.iloc[:, 1:-1]
y = data.iloc[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)'''

'train, valid = train_test_split(data, test_size=0.2)\n\ny_train = train.iloc[:, 0] \ny_valid = valid.iloc[:, 0]\nX_train = train.iloc[:, 1:-1]\nX_valid = valid.iloc[:, 1:-1]'

In [ ]:
#using pandas to split the dataset
train_data = data.sample(frac=0.8, random_state=42)  
test_data = data.drop(train_data.index)   

X_train = train_data.iloc[:, 1:-1]
y_train = train_data.iloc[:, 0]

X_test = test_data.iloc[:, 1:-1]
y_test = test_data.iloc[:, 0]


314

In [19]:
#using sckit-learn to normalize input data
'''from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_normalized = scaler.fit_transform(X_train)

print(X_train_normalized)'''

'from sklearn.preprocessing import StandardScaler\n\nscaler = StandardScaler()\n\nX_train_normalized = scaler.fit_transform(X_train)\n\nprint(X_train_normalized)'

In [21]:
#using tensorflow to normalize input data
import tensorflow as tf

def norm(input_data):
    normalizer = tf.keras.layers.Normalization()
    normalizer.adapt(np.array(input_data))
    data_normalized = normalizer(np.array(input_data))
    return data_normalized

X = norm(X_train)


In [34]:
#creation of a simple neural network with two hidden layers and the output layer with linear activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=20, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=1, activation='linear'))

In [35]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 20)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 381 (1.49 KB)

 Trainable params: 381 (1.49 KB)

 Non-trainable params: 0 (0.00 B)